In [1]:
import numpy as np
import pandas as pd

class BinomialAssetPricingModel:
    """
    A class to calculate various quantities related to a binomial asset pricing model.
    """
    def __init__(self, S0: float, u: float, d: float, r: float, n: int, K: float):
        """
        Initialize the binomial asset pricing model.
        
        Parameters:
            S0 (float): Initial asset price
            u (float): Up factor
            d (float): Down factor
            r (float): Risk-free rate
            n (int): Number of time steps
            K (float): Strike price
        """
        self.S0 = S0
        self.u = u
        self.d = d
        self.r = r
        self.n = n 
        self.K = K
        self.p = (1 + self.r - self.d) / (self.u - self.d)
        
    def asset_price(self, i: int) -> float:
        """
        Calculate the asset price at a given time step.
        
        Parameters:
            i (int): Time step
        
        Returns:
            float: Asset price
        """
        return self.S0 * (self.u ** i) * (self.d ** (self.n - i))

    def full_asset_price(self) -> pd.DataFrame:
        """
        Calculate the full asset price for each node in the binomial tree.
        
        Returns:
            pd.DataFrame: Asset prices
        """
        v = [[0.0 for _ in range(self.n + 1)] for _ in range(self.n + 1)]
        for i in range(self.n+1):
            v[self.n][i] = self.asset_price(i)

        for j in range(self.n - 1, -1, -1):
            for i in range(j + 1):
                v[j][i] = (self.d*v[j + 1][i + 1] + self.u* v[j + 1][i])/2

        return pd.DataFrame(v)

    def buy_option_price(self) -> pd.DataFrame:
        """
        Calculate the option price for buying the asset.
        
        Returns:
            pd.DataFrame: Option prices
        """
        v = [[0.0 for _ in range(self.n + 1)] for _ in range(self.n + 1)]

        for i in range(self.n + 1):
            v[self.n][i] = max(0.0, self.asset_price(i) - self.K)

        for j in range(self.n - 1, -1, -1):
            for i in range(j + 1):
                v[j][i] = (1 / (1 + self.r)) * (self.p * v[j + 1][i + 1] + (1 - self.p) * v[j + 1][i])

        return pd.DataFrame(v)

    def sell_option_price(self) -> pd.DataFrame:
        """
        Calculate the option price for selling the asset.
        
        Returns:
            pd.DataFrame: Option prices
        """
        v = [[0.0 for _ in range(self.n + 1)] for _ in range(self.n + 1)]

        for i in range(self.n + 1):
            v[self.n][i] = max(0.0, self.K - self.asset_price(i))

        for j in range(self.n - 1, -1, -1):
            for i in range(j + 1):
                v[j][i] = (1 / (1 + self.r)) * (self.p * v[j + 1][i + 1] + (1 - self.p) * v[j + 1][i])

        return pd.DataFrame(v)

    def shares_of_stock(self) -> pd.DataFrame:
        """
        Calculate the number of shares of stock held at each node in the binomial tree.
        
        Returns:
            pd.DataFrame: Shares of stock
        """
        v = [[0.0 for _ in range(self.n + 1)] for _ in range(self.n + 1)]
        buy_option = self.buy_option_price()
        full_asset = self.full_asset_price()
        for j in range(self.n, -1, -1):
            for i in range(j):
                v[j-1][i] = (buy_option.iloc[j,i]-buy_option.iloc[j,i+1])/(full_asset.iloc[j,i]-full_asset.iloc[j,i+1])
        return pd.DataFrame(v)

    def money_in_bank(self) -> pd.DataFrame:
        """
        Calculate the money in the bank at each node in the binomial tree.
        
        Returns:
            pd.DataFrame: Money in the bank
        """
        v = [[0.0 for _ in range(self.n + 1)] for _ in range(self.n + 1)]
        buy_option = self.buy_option_price()
        full_asset = self.full_asset_price()
        shares = self.shares_of_stock()
        
        for j in range(len(v)):
            for i in range(len(v)):
                v[j][i] = buy_option.iloc[j,i] - full_asset.iloc[j,i]*shares.iloc[j,i]
        return pd.DataFrame(v)

In [2]:
# Create an instance of the class
S0=100
u=1.07
d=1/u
r=0.01
n=3
K=100
model = BinomialAssetPricingModel(S0, u, d, r, n, K)

# Call the asset_price method to calculate the price of the asset at time step n, i=n
asset_price = model.full_asset_price()
print(f'Asset price: \n{pd.DataFrame(asset_price)}\n')

# Call the option_price method to calculate the price of the option with strike price K=100 at time all
buy_option_price = model.buy_option_price()
print(f'Buy Option price (European Options): \n{pd.DataFrame(buy_option_price)}\n')

sell_option_price = model.sell_option_price()
print(f'Sell Option price (American Options): \n{pd.DataFrame(sell_option_price)}\n')

stock = model.shares_of_stock()
print(f"Share of stock: \n{stock}\n")

bank = model.money_in_bank()
print(f"Money invested in bank \n{bank}\n")

Asset price: 
            0           1       2         3
0  100.000000    0.000000    0.00    0.0000
1   93.457944  107.000000    0.00    0.0000
2   87.343873  100.000000  114.49    0.0000
3   81.629788   93.457944  107.00  122.5043

Buy Option price (European Options): 
          0          1          2        3
0  6.574383   0.000000   0.000000   0.0000
1  2.128460  10.229334   0.000000   0.0000
2  0.000000   3.859951  15.480099   0.0000
3  0.000000   0.000000   7.000000  22.5043

Sell Option price (American Options): 
           0         1    2    3
0   3.633398  0.000000  0.0  0.0
1   6.700121  1.258939  0.0  0.0
2  11.666028  2.869852  0.0  0.0
3  18.370212  6.542056  0.0  0.0

Share of stock: 
          0         1    2    3
0  0.598201  0.000000  0.0  0.0
1  0.304987  0.801943  0.0  0.0
2 -0.000000  0.516908  1.0  0.0
3  0.000000  0.000000  0.0  0.0

Money invested in bank 
           0         1          2        3
0 -53.245734   0.00000   0.000000   0.0000
1 -26.374975 -75.5